## Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
training = pd.read_csv("fer2013train.csv")
test = pd.read_csv("fer2013test.csv")
full = pd.concat([training,test])

In [ ]:
import itertools
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
full = full[full.emotion != 1]
zeros = full[full.emotion == 0]
oneHigh = full[full.emotion != 0]
oneHigh.loc[:,'emotion'] -= 1
full = pd.concat([zeros, oneHigh])

In [ ]:
full = full.sample(frac=.1)

In [ ]:
y = full.emotion
y.value_counts().plot(kind='bar',title ='FER Unbalanced')

In [ ]:
c0, c1,c2,c3,c4,c5 = y.value_counts()
data0 = full[y==0].sample(c5)
data1 = full[y==1].sample(c5)
data2 = full[y==2].sample(c5)
data3 = full[y==3].sample(c5)
data4 = full[y==4].sample(c5)
data5 = full[y==5].sample(c5)

fullUnder = pd.concat([data0, data1,data2,data3,data4,data5],axis=0)
fullUnder.describe()

In [ ]:
y = fullUnder.emotion
x = fullUnder.pixels
x = pd.DataFrame(x.str.split().values.tolist())
y.value_counts().plot(kind='bar',title='FER Balanced')

In [ ]:
medium = fullUnder.sample(frac=.1)

In [ ]:
my = medium.emotion
mx = pd.DataFrame(medium.pixels.str.split().values.tolist())

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(x, y, random_state = 0)
train_xm, val_xm, train_ym, val_ym = train_test_split(mx, my, random_state = 0)

**Scaled data**

In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(train_x) 
scaler.fit(train_xm)
strain_x = scaler.transform(train_x)
strain_xm = scaler.transform(train_xm)
# apply same transformation to test data
sval_x = scaler.transform(val_x)  
sval_xm = scaler.transform(val_xm)

 ## Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy')
title = "Decision Tree (no pruning)"
plot_learning_curve(tree,title,train_x, train_y, ylim=(0.1,1.01),cv=5,n_jobs=4)

In [ ]:
from sklearn.model_selection import cross_val_score
crossVal = cross_val_score(tree, train_x, train_y, cv=5)
print("Unpruned:",np.mean(crossVal))

In [ ]:
for i in range(1,15):
    print(i)
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=i)
    title = "Decision Tree of Max Depth: " + str(i)
    plot_learning_curve(tree,title, train_x, train_y, ylim=(0.1, 1.01), cv=5, n_jobs=4)
    crossVal = cross_val_score(tree, train_x, train_y, cv=5)
    print("Pruned:",i,np.mean(crossVal))

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy',max_depth=7)
pred_y = tree.fit(train_x, train_y).predict(val_x)
cnf_matrix = confusion_matrix(val_y, pred_y)
plt.figure()
plot_confusion_matrix(cnf_matrix, 
                      classes=['Angry','Fear','Happy','Sad','Surprise','Neutral'],
                      normalize=True, title='Decision Tree Confusion Matrix')
print("F1 Score", f1_score(pred_y, val_y,average='weighted'))
print("Score", tree.score(val_x, val_y))

**Sizes**

In [ ]:
title = "Decision Tree (n=245)"
plot_learning_curve(tree,title, train_xm, train_ym, ylim=(0.0, 1.01), cv=5, n_jobs=4)

## Neural Networks

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [ ]:
title = "Default Neural Network"
nn = MLPClassifier(solver='lbfgs',random_state=1,early_stopping=True)
plot_learning_curve(nn,title, strain_x, train_y, ylim=(0.0, 1.01), cv=5, n_jobs=4)

In [ ]:
crossVal = cross_val_score(nn, strain_x, train_y, cv=5)
print("Untuned:",np.mean(crossVal))

**Hyperparameter Tuning**

In [ ]:
params = {'solver':['adam','sgd','lbfgs'],'alpha': 10**-np.arange(1,7,dtype=float),
          'learning_rate':['constant','adaptive']}
search = GridSearchCV(estimator=nn, param_grid=params,cv=3)
search.fit(strain_x,train_y)

In [ ]:
title = "Neural Network After Tuning Hyperparameters"
tunedNN = MLPClassifier(solver='lbfgs',random_state=1,alpha=0.1)
param1 = {'hidden_layer_sizes': range(200,4201,1000)}
param2 = {'hidden_layer_sizes': [(x,100) for x in range(200,4201,1000)]}
rSearch1 = GridSearchCV(estimator=tunedNN, 
                            param_grid=[param1,param2])
rSearch1.fit(strain_x, train_y)

In [ ]:
tunedNN = search.best_estimator_
print(tunedNN)
plot_learning_curve(tunedNN, title, strain_x, train_y, ylim=(0.0, 1.01), cv=5, n_jobs=4)
crossVal = cross_val_score(tunedNN, strain_x, train_y, cv=5)
print("Tuned:",np.mean(crossVal))

**Scores**

In [ ]:
pred_y = tunedNN.fit(strain_x, train_y).predict(val_X)
cnf_matrix = confusion_matrix(sval_y, pred_y)
plt.figure()
plot_confusion_matrix(cnf_matrix, 
                      classes=['Angry','Fear','Happy','Sad','Surprise','Neutral'],
                      normalize=True, title='Neural Network Confusion Matrix')
print("Score", tunedNN.score(sval_X, val_y))

**Sizes**

In [ ]:
title = "Neural Network (n=245)"
plot_learning_curve(tree,title, strain_xm, train_ym, ylim=(0.0, 1.01), cv=5, n_jobs=4)

## Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
bdt = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',max_depth=1))
bdt.fit(train_x, train_y)
title = "Default AdaBoost"
plot_learning_curve(bdt,title, train_x, train_y, ylim=(0.1, 1.01), cv=5, n_jobs=4)
crossVal = cross_val_score(bdt, train_x, train_y, cv=5)
print("Untuned",np.mean(crossVal))

**Hyperparameter Tuning**

In [ ]:
params = {'learning_rate': np.logspace(-3,2,3),'n_estimators':range(10,1000,100)}
search = GridSearchCv(estimator=bdt, param_grid=params,scoring='f1')
search.fit(train_x,train_y)

In [ ]:
title = "AdaBoost After Tuning Hyperparameters"
bdt = search.best_estimator_
plot_learning_curve(bdt, title, train_X, train_y, ylim=(0.0, 1.01), cv=5, n_jobs=4)
crossVal = cross_val_score(bdt, train_X, train_y, cv=5)
print("Tuned:",np.mean(crossVal))

**Scores**

In [ ]:
bdt = search.best_estimator_
pred_y = bdt.fit(train_x, train_y).predict(val_X)
cnf_matrix = confusion_matrix(sval_y, pred_y)
plt.figure()
plot_confusion_matrix(cnf_matrix, 
                      classes=['Angry','Fear','Happy','Sad','Surprise','Neutral'],
                      normalize=True, title='AdaBoost Confusion Matrix')
print("Score", bdt.score(sval_X, val_y))

**Sizes**

In [ ]:
title = "AdaBoost (n=245)"
plot_learning_curve(bdt,title, strain_xm, train_ym, ylim=(0.0, 1.01), cv=5, n_jobs=4)

## Support Vector Machine

In [ ]:
from sklearn import svm
svmClf = svm.SVC()
svmClf.fit(strain_x, train_y)
title = "Default SVM"
plot_learning_curve(svmClf,title, strain_x, train_y, ylim=(0.5, 1.01), cv=5, n_jobs=4)
crossVal = cross_val_score(svmClf, strain_x, train_y, cv=5)
print("Untuned",np.mean(crossVal))

**Hyperparameter Tuning**

In [ ]:
params = {'gamma': np.logspace(-5,-3,3),'C':np.logspace(0,2,3),'kernel':['linear','rbf']}
search = GridSearchCV(estimatro=svmClf,param_grid=params)
search.fit(strain_x,train_y)

In [ ]:
title = "SVM After Tuning Hyperparameters"
svmClf = search.best_estimator_
plot_learning_curve(svmClf, title, strain_x, train_y, ylim=(0.0, 1.01), cv=5, n_jobs=4)
crossVal = cross_val_score(bdt, strain_x, train_y, cv=5)
print("Tuned:",np.mean(crossVal))

**Scores**

In [ ]:
svmClf = search.best_estimator_
pred_y = svmClf.fit(strain_x, train_y).predict(sval_x)
cnf_matrix = confusion_matrix(val_y, pred_y)
plt.figure()
plot_confusion_matrix(cnf_matrix, 
                      classes=['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral'],
                      normalize=True, title='SVM Confusion Matrix')
print("Score", svmClf.score(sval_x, val_y))

**Sizes**

In [ ]:
title = "SVM (n=245)"
plot_learning_curve(svmClf,title, strain_xm, train_ym, ylim=(0.0, 1.01), cv=5, n_jobs=4)

## k-Nearest Neighbors

In [ ]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier()
knn.fit(strain_x, train_y)
title = "Default k-NN"
plot_learning_curve(knn,title, strain_x, train_y, ylim=(0.0, 1.01), cv=5, n_jobs=4)
crossVal = cross_val_score(knn, strain_x, train_y, cv=5)
print("Untuned",np.mean(crossVal))

**Hyperparameter Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'n_neighbors': range(1,111,10), 'weights':['uniform','distance'], 'p': range(1,3)}
gs = GridSearchCV(estimator=knn, param_grid=params,cv=3,scoring='f1_weighted')
gs.fit(strain_x, train_y)
print(gs)

In [ ]:
title = "k-NN After Tuning"
knn = gs.best_estimator_
plot_learning_curve(knn, title, strain_x, train_y, ylim=(0.0, 1.01), cv=5, n_jobs=4)
crossVal = cross_val_score(knn, strain_x, train_y, cv=5)
print("Tuned:",np.mean(crossVal))

**Scores**

In [ ]:
knn = search.best_estimator_
pred_y = knn.fit(strain_x, train_y).predict(sval_x)
cnf_matrix = confusion_matrix(sval_y, pred_y)
plt.figure()
plot_confusion_matrix(cnf_matrix, 
                      classes=['Angry','Fear','Happy','Sad','Surprise','Neutral'],
                      normalize=True, title='k-NN Confusion Matrix')
print("Score", knn.score(sval_X, val_y))

**Sizes**

In [ ]:
title = "kNN (n=245)"
plot_learning_curve(knn,title, strain_xm, train_ym, ylim=(0.0, 1.01), cv=5, n_jobs=4)